In [1]:
import numpy as np

from qibo.symbols import Z
from qibo import hamiltonians

from qiboml.models.encoding_decoding import *
from qiboml.models.ansatze import *

def run_model(model, x):
    for layer in model:
        x = layer.forward(x)
    return x

In [2]:
# binary data
model = [BinaryEncodingLayer(nqubits=3), ReuploadingLayer(nqubits=3, qubits=(0,2)), QuantumDecodingLayer(nqubits=3, qubits=(1,0))]
data = np.array([[0,0,0], [0,1,0], [1,1,0]])

for x in data:
    print(run_model(model, x).probabilities())


tf.Tensor([0. 0. 0. 0. 1. 0. 0. 0.], shape=(8,), dtype=float64)
tf.Tensor([0. 0. 1. 0. 0. 0. 0. 0.], shape=(8,), dtype=float64)
tf.Tensor([0. 0. 0. 0. 1. 0. 0. 0.], shape=(8,), dtype=float64)


In [3]:
# real data
nqubits = 4
observable = hamiltonians.SymbolicHamiltonian(np.prod([Z(q) for q in range(nqubits)]))
model = [PhaseEncodingLayer(nqubits, qubits=range(3)), ReuploadingLayer(nqubits), ExpectationLayer(nqubits, observable=observable)]
data = np.pi * np.random.randn(3**2).reshape(3, 3)

for x in data:
    print(run_model(model, x))

[Qibo 0.2.6|INFO|2024-06-20 12:12:39]: Using qibojit (numba) backend on /CPU:0


tf.Tensor(1.0, shape=(), dtype=float64)
tf.Tensor(1.0, shape=(), dtype=float64)
tf.Tensor(1.0, shape=(), dtype=float64)


In [4]:
# pytorch interface
import torch
import qiboml.models.pytorch as pt

model = torch.nn.Sequential(
    torch.nn.Linear(128,5),
    torch.nn.Sigmoid(),
    pt.BinaryEncodingLayer(5),
    pt.ReuploadingLayer(nqubits=5, qubits=(0,2,4)),
    pt.QuantumDecodingLayer(nqubits=5, qubits=reversed(range(5)))
)

print(f"> Model: {model}")
print("> Parameters")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.shape)

data = torch.randn(3,128)
print("> Outputs")
for x in data:
    print(model(x).probabilities())


> Model: Sequential(
  (0): Linear(in_features=128, out_features=5, bias=True)
  (1): Sigmoid()
  (2): BinaryEncodingLayer(nqubits=5, qubits=[0, 1, 2, 3, 4], circuit=<qibo.models.circuit.Circuit object at 0x35734a490>, initial_state=None, backend=tensorflow)
  (3): ReuploadingLayer(nqubits=5, qubits=(0, 2, 4), circuit=<qibo.models.circuit.Circuit object at 0x354a67990>, initial_state=None, backend=tensorflow)
  (4): QuantumDecodingLayer(nqubits=5, qubits=<range_iterator object at 0x357448d20>, circuit=<qibo.models.circuit.Circuit object at 0x35131c990>, initial_state=None, backend=tensorflow, nshots=1000)
)
> Parameters
0.weight torch.Size([5, 128])
0.bias torch.Size([5])
3.ReuploadingLayer torch.Size([6, 1])
> Outputs
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(32,), dtype=float64)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(32,), dtype=fl

In [5]:
# keras/tensorflow interface
import keras
import tensorflow as tf
import qiboml.models.keras as ks
import qiboml.models.encoding_decoding as ed
import qiboml.models.ansatze as ans

nqubits = 5
observable = hamiltonians.SymbolicHamiltonian(np.prod([Z(q) for q in range(nqubits)]))

q_model = ks.Model(
    layers = [
        ed.BinaryEncodingLayer(5),
        ans.ReuploadingLayer(nqubits=5, qubits=(0,2,4)),
        ed.ExpectationLayer(nqubits=5, qubits=reversed(range(5)), observable=observable)
    ]
)

model = keras.Sequential([
    keras.layers.Dense(5, activation="sigmoid", name="dense"),
    q_model,
])


print(f"> Model: {model}")

data = tf.random.uniform((3,128))
print("> Outputs")
for x in data:
    x = tf.expand_dims(x, axis=0)
    print(model(x).probabilities())



> Model: <Sequential name=sequential, built=False>
> Outputs


OperatorNotAllowedInGraphError: Exception encountered when calling Model.call().

[1mIterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.[0m

Arguments received by Model.call():
  • args=('<KerasTensor shape=(1, 5), dtype=float32, sparse=False, name=keras_tensor_1>',)
  • kwargs=<class 'inspect._empty'>